In [1]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 23160, done.
remote: Counting objects: 100% (622/622), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 23160 (delta 436), reused 570 (delta 407), pack-reused 22538
Receiving objects: 100% (23160/23160), 29.55 MiB | 19.84 MiB/s, done.
Resolving deltas: 100% (17129/17129), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be376084 version check against PyTorch's CUDA version


In [2]:
# mount google drive where the zipped file of images located
from google.colab import drive
drive.mount('/content/gdrive')

# unzip the file
!unzip "/content/gdrive/My Drive/OleanderDataset.zip"

Mounted at /content/gdrive
Archive:  /content/gdrive/My Drive/OleanderDataset.zip
   creating: Oleander Plant/
   creating: Oleander Plant/Images/
  inflating: Oleander Plant/Images/OleanderCNN00001.png  
  inflating: Oleander Plant/Images/OleanderCNN00002.png  
  inflating: Oleander Plant/Images/OleanderCNN00003.png  
  inflating: Oleander Plant/Images/OleanderCNN00004.png  
  inflating: Oleander Plant/Images/OleanderCNN00005.png  
  inflating: Oleander Plant/Images/OleanderCNN00006.png  
  inflating: Oleander Plant/Images/OleanderCNN00007.png  
  inflating: Oleander Plant/Images/OleanderCNN00008.png  
  inflating: Oleander Plant/Images/OleanderCNN00009.png  
  inflating: Oleander Plant/Images/OleanderCNN00010.png  
  inflating: Oleander Plant/Images/OleanderCNN00011.png  
  inflating: Oleander Plant/Images/OleanderCNN00012.png  
  inflating: Oleander Plant/Images/OleanderCNN00013.png  
  inflating: Oleander Plant/Images/OleanderCNN00014.png  
  inflating: Oleander Plant/Images/Oleand

In [3]:
#Initialisation of dataset and CNN structure



# define the dataset
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image

class NNDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to 
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "Images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "Masks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "Images", self.imgs[idx])
        mask_path = os.path.join(self.root, "Masks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # convert masks to grayscale mode to distinguish background and objects
        mask = Image.open(mask_path).convert("L")

        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

# import Mask R-CNN architecture for fine-tuning
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model


# determine the version of cuda
!cat /usr/local/cuda/version.txt

# import helper functions
from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
        # NB: may add rotation and resize operations
    return T.Compose(transforms)


# use our dataset and defined transformations
dataset = NNDataset('Oleander Plant', get_transform(train=True))
dataset_test = NNDataset('Oleander Plant', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
print(len(dataset))
# NB: change the portion of train and test set according to the size of the whole dataset
dataset = torch.utils.data.Subset(dataset, indices[:-6])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-6:])
print(len(dataset))

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)




CUDA Version 11.0.228
86
80


In [ ]:
device = torch.device('cuda') #if torch.cuda.is_available() else torch.device('cpu')
#device = torch.device('cpu')
# our dataset has two classes only - background and label
num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [5]:
# CNN training
# train for epochs
# NB: change number of epochs according to the size of dataset
# few epochs for small dataset to avoid overfitting
num_epochs = 2

for epoch in range(num_epochs):
    # train for one epoch, printing every 5 iteration
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=5)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [ 0/40]  eta: 0:42:28  lr: 0.000133  loss: 11.2139 (11.2139)  loss_classifier: 0.3593 (0.3593)  loss_box_reg: 0.0883 (0.0883)  loss_mask: 10.7315 (10.7315)  loss_objectness: 0.0267 (0.0267)  loss_rpn_box_reg: 0.0082 (0.0082)  time: 63.7161  data: 14.2320  max mem: 0


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-47382ee65eb2>", line 9, in <module>
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=5)
  File "/content/engine.py", line 46, in train_one_epoch
    losses.backward()
  File "/usr/local/lib/python3.7/dist-packages/torch/tensor.py", line 245, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py", line 147, in backward
    allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    s

KeyboardInterrupt: ignored

In [ ]:
#save model
import torch
import torch.onnx as onnx
import torchvision.models as models





In [ ]:
# test
# pick one image from the test set
img, _ = dataset_test[1]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])
  
prediction

In [ ]:
# view verification image
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy()).convert('L')

In [ ]:
# view prediction
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy()).convert('L')

In [ ]:
while True:pass